

<p align="center">
  <img src="https://github.com/Owaiskhan9654/Granite-4.0-Fine-Tuning/blob/main/ibm-granite-4-0-release.jpeg?raw=true" 
       alt="Granite 4.0"
       width="450">
</p>

# **Install dependencies**

In [1]:
import os, sys

os.environ["XLA_FLAGS"] = "--xla_cpu_enable_fast_math=false"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"   # also suppresses XLA logs
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

sys.stderr = open(os.devnull, "w")

In [2]:
!pip install -q --no-deps transformers accelerate datasets peft evaluate sentencepiece safetensors bitsandbytes protobuf==3.20.1 mlflow dagshub appdirs databricks-agents huggingface_hub anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 17.0 MB/s eta 0:00:00


In [3]:
import json
import torch
import dagshub
import mlflow
import warnings

import pandas as pd
import sys, os, json
from pydantic._internal._generate_schema import UnsupportedFieldAttributeWarning
warnings.filterwarnings("ignore", category=UnsupportedFieldAttributeWarning)
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, create_repo
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling,TrainerCallback


from peft import LoraConfig, get_peft_model
from peft import PeftModel

# **Configurations**

In [4]:
# -----------------------
# Model trainings config
# -----------------------
MODEL_NAME = "ibm-granite/granite-4.0-micro"
DATA_PATH = "/kaggle/working/data.jsonl"   # upload your jsonl here or change path
OUTPUT_DIR = "/kaggle/working/granite-finetuned"
TRAIN_BATCH_SIZE = 4
GRAD_ACCUM = 8
EPOCHS = 10
LEARNING_RATE = 2e-4
MAX_LENGTH = 1024

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
user_secrets = UserSecretsClient()


os.environ["DAGSHUB_USERNAME"] = "owais0425"
os.environ["DAGSHUB_TOKEN"] = user_secrets.get_secret("DAGSHUB_TOKEN")# created from https://dagshub.com/user/settings/tokens
os.environ["DAGSHUB_API_USE_OAUTH"] = "false"
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow"


mlflow_token = user_secrets.get_secret("DAGSHUB_TOKEN")
dagshub.auth.add_app_token(mlflow_token)   # stores token in client cache 
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

In [6]:
dagshub.init(repo_owner="owais0425",repo_name="Granite-4.0-Fine-Tuning",mlflow=True)

Accessing as owais0425

Initialized MLflow to track repo "owais0425/Granite-4.0-Fine-Tuning"

Repository owais0425/Granite-4.0-Fine-Tuning initialized!

In [7]:
class MlflowCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return
        step = getattr(state, "global_step", None) or 0
        for k, v in logs.items():
            # only log numeric metrics
            try:
                val = float(v)
            except Exception:
                continue
            mlflow.log_metric(k, val, step=step)

# **Data Preperations**

### **Example data format (instruction tuning)**


* {"instruction":"Summarize the following: ...", "input":"Long document ...", "output":"Short summary..."}
* {"instruction":"Translate to French", "input":"Hello", "output":"Bonjour"}


https://www.kaggle.com/code/owaiskhan9654/fairytaleqa-data-prep-granite-4-0-fine-tuning/output

In [8]:

CSV_PATH = "/kaggle/input/fairytaleqa-data-prep-granite-4-0-fine-tuning/FairyTaleQA_processed_instruct_dataset.csv"
JSONL_PATH = "/kaggle/working/FairyTaleQA_processed_instruct_dataset.jsonl"


if os.path.exists(CSV_PATH):
    print("CSV found, converting to JSONL...")

    df = pd.read_csv(CSV_PATH)

    with open(JSONL_PATH, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            if _==150:
                break
            obj = {
                "instruction": row.get("instruction", ""),
                "input": row.get("input", ""),
                "output": row.get("output", "")
            }
            f.write(json.dumps(obj, ensure_ascii=False) + "\n")

    DATA_PATH = JSONL_PATH
    print("Converted JSONL saved at:", DATA_PATH)


else:
    print("CSV not found — creating fallback JSONL...")

    fallback = [
        {"instruction": "Summarize the text.", "input": "Kaggle free GPUs are T4s.", "output": "Kaggle provides free T4 GPUs with limits."},
        {"instruction": "Translate to French.", "input": "Hello", "output": "Bonjour"},
    ]

    fallback_path = "/kaggle/working/fallback_data.jsonl"
    with open(fallback_path, "w", encoding="utf-8") as f:
        for ex in fallback:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")

    DATA_PATH = fallback_path
    print("Using fallback JSONL:", DATA_PATH)

print("\nLoading dataset:", DATA_PATH)

records = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

print("Loaded examples:", len(records))
print("Sample example:", records[0])


CSV found, converting to JSONL...
Converted JSONL saved at: /kaggle/working/FairyTaleQA_processed_instruct_dataset.jsonl

Loading dataset: /kaggle/working/FairyTaleQA_processed_instruct_dataset.jsonl
Loaded examples: 150
Sample example: {'instruction': 'why did setanta see his father little?', 'input': "the-boyhood-of-cuchulain: of his father he saw little. his mind had become impaired, and he was confined in a secluded part of the dun. but whenever he spoke to dectera of what was nearest his heart, and his desire to enter the military school at emain macha, she laughed, and said that he was not yet old enough to endure that rough life. but secretly she was alarmed, and formed plans to detain him at home altogether. then setanta concealed his desire, but enquired narrowly concerning the partings of the roads on the way to emania. at last, when he was ten years old, selecting a favourable night, setanta stole away from his father's dun, and before morning had crossed the frontier. he th

# **Tokenizer & dataset loading**

In [9]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading dataset (jsonl)...")
# Robust loader for jsonlines
raw_ds = load_dataset("json", data_files={"train": DATA_PATH}, split="train")

# Build single text prompt for SFT; adjust template if you prefer a different format.
def build_prompt(example):
    instr = example.get("instruction", "")
    inp = example.get("input", "")
    out = example.get("output", "")
    prompt = f"### Instruction:\n{instr}\n\n### Input:\n{inp}\n\n### Response:\n{out}"
    return prompt

raw_ds = raw_ds.map(lambda ex: {"text": build_prompt(ex)}, remove_columns=raw_ds.column_names)

# Tokenize
print("Tokenizing...")
def tokenize_fn(batch):
    toks = tokenizer(batch["text"], truncation=True, max_length=MAX_LENGTH, padding="max_length")
    toks["labels"] = toks["input_ids"].copy()
    
    return toks

train_ds = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"]) 
print("Prepared dataset — examples:", len(train_ds))


Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Tokenizing...


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Prepared dataset — examples: 150


# **Model load (BitsAndBytesConfig / fallback)**

In [10]:
print("Preparing model load — trying bitsandbytes 4-bit first if available...")
use_bnb = False
quant_config = None
try:
    import bitsandbytes as bnb  # noqa: F401
    from transformers import BitsAndBytesConfig
    if torch.cuda.is_available():
        use_bnb = True
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
    else:
        print("bitsandbytes installed but CUDA not available — using fp16 fallback.")
except Exception as e:
    print("bitsandbytes import failed or not installed — using fp16 fallback. Error:", str(e))

if use_bnb and quant_config is not None:
    print("Loading model with quantization_config (4-bit)...")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", quantization_config=quant_config)
else:
    print("Fallback load: loading model in fp16 on GPU if available (no bitsandbytes).")
    device_map = "auto" if torch.cuda.is_available() else None
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32), device_map=device_map)

print("Model loaded — device map:", getattr(model, "hf_device_map", getattr(model, "device_map", None)))

Preparing model load — trying bitsandbytes 4-bit first if available...
Loading model with quantization_config (4-bit)...


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded — device map: {'model.embed_tokens': 0, 'lm_head': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 1, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.layers.32': 1, 'model.layers.33': 1, 'model.layers.34': 1, 'model.layers.35': 1, 'model.layers.36': 1, 'model.layers.37': 1, 'model.layers.38': 1, 'model.layers.39': 1, 'model.norm': 1, 'model.rotary_emb': 1}


# **Apply LoRA (PEFT)** (Quantization)

In [11]:
print("Applying LoRA adapters (PEFT)...") # low rank adaptations
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],# "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.config.use_cache = False
print("PEFT/LoRA applied. Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))


Applying LoRA adapters (PEFT)...
PEFT/LoRA applied. Trainable params: 2621440


# **Training setup & run (Trainer)**

In [12]:
# start MLflow run and attach callback

EXPERIMENT_NAME = "granite-4.0-finetuning-v1-ashutosh"
RUN_NAME = f"granite4-micro-bs{TRAIN_BATCH_SIZE}-lr{LEARNING_RATE}-v3"

mlflow.set_experiment(EXPERIMENT_NAME)
EXPERIMENT_TAG = "granite-4.0-finetuning"


with mlflow.start_run(run_name=RUN_NAME):
    mlflow.set_tag("experiment", EXPERIMENT_TAG)
    mlflow.set_tag("mlflow.source.name", "kaggle_notebook_Granite_4_0_Fine_tuning")
    mlflow.set_tag("mlflow.source.type", "KAGGLE_NOTEBOOK")
    mlflow.set_tag("project", "granite-finetuning")
    
    mlflow.log_param("train_batch_size", TRAIN_BATCH_SIZE)
    mlflow.log_param("grad_accum", GRAD_ACCUM)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model_name", MODEL_NAME)

    print("Setting up Trainer...")
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=TRAIN_BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=torch.cuda.is_available(),
        logging_steps=2,
        save_steps=1,
        save_total_limit=3,
        remove_unused_columns=False,
        report_to="none",
        label_names=["labels"]
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        data_collator=data_collator,
    )

    trainer.add_callback(MlflowCallback())

    print("Starting training — this may take a while depending on data and GPU resources.")
    trainer.train()

    # save & log model artifact
    trainer.save_model(OUTPUT_DIR)
    mlflow.log_artifact(OUTPUT_DIR)


Setting up Trainer...
Starting training — this may take a while depending on data and GPU resources.


Step,Training Loss
2,3.076300
4,2.847100
6,2.666900
8,2.586900
10,2.460600
12,2.330900
14,2.290500
16,2.108800
18,2.089600
20,1.873500


🏃 View run granite4-micro-bs4-lr0.0002-v3 at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2/runs/7466779510c3430987c1a9ce98b2fd30
🧪 View experiment at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2


**All trainings experiments metrics in ML flow can be tracked here** - https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2?viewStateShareKey=3894e7dac091113a949e1a0b144bdfbf23f857b1cfb2b6251e919052fe25b155&compareRunsMode=TABLE****

# **Save adapters + tokenizer**

In [13]:

print("Saving LoRA adapter and tokenizer to:", OUTPUT_DIR)
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved.")

Saving LoRA adapter and tokenizer to: /kaggle/working/granite-finetuned
Saved.


In [14]:
model = model.merge_and_unload()   # returns merged HF model (if using peft>=0.5.x)

In [15]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Saved FULL merged model to:", OUTPUT_DIR)

Saved FULL merged model to: /kaggle/working/granite-finetuned


In [16]:
HF_TOKEN = user_secrets.get_secret("Hugging_Face_model_Push_Secret")

REPO_ID = "owaiskha9654/granite-4-finetuned-FairyTaleQA"
LOCAL_DIR = OUTPUT_DIR

# # Creating repository if not exists
# create_repo(REPO_ID, token=HF_TOKEN, private=False)

# Uploading entire folder (model + LoRA + tokenizer)
api = HfApi()
api.upload_folder(
    folder_path=LOCAL_DIR,
    repo_id=REPO_ID,
    token=HF_TOKEN,
    commit_message="Upload fine-tuned model and LoRA adapter"
)

print("Uploaded to Hugging Face:", REPO_ID)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploaded to Hugging Face: owaiskha9654/granite-4-finetuned-FairyTaleQA


# **Quick inference test**

In [17]:
base = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
peft_model = PeftModel.from_pretrained(base, OUTPUT_DIR)
peft_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GraniteMoeHybridForCausalLM(
      (model): GraniteMoeHybridModel(
        (embed_tokens): Embedding(100352, 2560, padding_idx=100256)
        (layers): ModuleList(
          (0-39): 40 x GraniteMoeHybridDecoderLayer(
            (input_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
            (post_attention_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
            (shared_mlp): GraniteMoeHybridMLP(
              (activation): SiLU()
              (input_linear): Linear(in_features=2560, out_features=16384, bias=False)
              (output_linear): Linear(in_features=8192, out_features=2560, bias=False)
            )
            (self_attn): GraniteMoeHybridAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=2560, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
            

**All trainings experiments metrics in ML flow can be tracked here** - https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2?viewStateShareKey=3894e7dac091113a949e1a0b144bdfbf23f857b1cfb2b6251e919052fe25b155&compareRunsMode=TABLE

**Its basically a Quantization aware training post Callibration**

In [18]:
prompt = "### Instruction:\nSummarize the text.\n\n### Input:\nKaggle T4 GPU is a powerful tool for data scientists and machine learning enthusiasts. It provides access to Google's Tensor Processing Units (TPUs) for running machine learning models and performing data analysis.\n\n ### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(next(peft_model.parameters()).device)
out = peft_model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(out[0], skip_special_tokens=True))

### Instruction:
Summarize the text.

### Input:
Kaggle T4 GPU is a powerful tool for data scientists and machine learning enthusiasts. It provides access to Google's Tensor Processing Units (TPUs) for running machine learning models and performing data analysis.

 ### Response:
Kaggle T4 GPU is a tool for data scientists and machine learning enthusiasts.


# **Evaluation**

LLM Evaluation Metrics
There are two types of LLM evaluation metrics:

**Heuristic-based metrics:** These metrics calculate a score for each data record (row in terms of Pandas/Spark dataframe), based on certain functions, such as: Rouge (mlflow.metrics.rougeL()), Flesch Kincaid (mlflow.metrics.flesch_kincaid_grade_level()) or Bilingual Evaluation Understudy (BLEU) (mlflow.metrics.bleu()). These metrics are similar to traditional continuous value metrics. For the list of built-in heuristic metrics and how to define a custom metric with your own function definition, see the Heuristic-based Metrics section.

**LLM-as-a-Judge metrics:** LLM-as-a-Judge is a new type of metric that uses LLMs to score the quality of model outputs. It overcomes the limitations of heuristic-based metrics, which often miss nuances like context and semantic accuracy. LLM-as-a-Judge metrics provides a more human-like evaluation for complex language tasks while being more scalable and cost-effective than human evaluation. MLflow provides various built-in LLM-as-a-Judge metrics and supports creating custom metrics with your own prompt, grading criteria, and reference examples. See the LLM-as-a-Judge Metrics section for more details.



In [19]:
CSV_PATH = "/kaggle/input/fairytaleqa-data-prep-granite-4-0-fine-tuning/FairyTaleQA_processed_instruct_dataset.csv"
JSONL_PATH = "/kaggle/working/FairyTaleQA_eval_groundtruth.jsonl"
START_INDEX = 150 

assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

rows = []
with open(JSONL_PATH, "w", encoding="utf-8") as fw:
    for idx, row in df.iterrows():
        if idx <= START_INDEX:
            continue
        if idx>300:
            break
        instr = row.get("instruction", "") or ""
        inp = row.get("input", "") or ""
        out = row.get("output", "") or ""
        # Build a combined prompt input (adjust formatting as needed)
        prompt = instr.strip()
        if inp.strip():
            prompt = prompt + "\n\n" + inp.strip()
        obj = {
            "instruction": instr,
            "input": inp,
            "output": out,
            # ground-truth format for MLflow / eval usage:
            "inputs": prompt,
            "ground_truth": out
        }
        fw.write(json.dumps(obj, ensure_ascii=False) + "\n")
        rows.append({"inputs": prompt, "ground_truth": out})

print(f"Saved evaluation JSONL to: {JSONL_PATH}")
print(f"Total eval rows written (index >= {START_INDEX}): {len(rows)}")

eval_df = pd.DataFrame(rows)
eval_df = eval_df[eval_df["ground_truth"].astype(str).str.strip() != ""].reset_index(drop=True)
print(f"Final eval_df rows after dropping empty ground-truths: {len(eval_df)}")

# Show head
eval_df.head()


Saved evaluation JSONL to: /kaggle/working/FairyTaleQA_eval_groundtruth.jsonl
Total eval rows written (index >= 150): 150
Final eval_df rows after dropping empty ground-truths: 150


,inputs,ground_truth
0,who was the wicked magician?\n\nprince milan's...,a mighty ruler in the underworld.
1,what did hyacinthia tell the prince to do when...,throw himself at once on the ground and approa...
2,how will the thirtieth duck feel when the prin...,grateful.
3,what happened when hyacinthia stamped on the g...,the earth opened and they both sank down into ...
4,why was the king and queen lamenting before th...,they did not want the prince to leave.


In [20]:
from anthropic import Anthropic
from mlflow.metrics import latency, toxicity
from mlflow.metrics.genai import answer_correctness


os.environ["ANTHROPIC_API_KEY"] = user_secrets.get_secret("anthropic_api_key") 

client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

In [21]:
def anthropic_model(inputs):
    outputs = []
    for q in inputs:
        resp = client.messages.create(
            model="claude-sonnet-4-5-20250929",   
            messages=[{"role": "user", "content": q}],
            max_tokens=512,
            temperature=0.0
        )
    
        try:
            text = resp.content[0].text
        except:
            text = str(resp)
        
        outputs.append(text)
    return outputs

In [22]:
mlflow.set_experiment("llm-based-evaluation")

EXPERIMENT_NAME = "anthropic-evaluation-v3"
RUN_NAME = f"granite4-anthropic-evaluation-v3"

mlflow.set_experiment(EXPERIMENT_NAME)
EXPERIMENT_TAG = "granite-4.0-finetuning"


with mlflow.start_run(run_name=RUN_NAME):
    mlflow.set_tag("experiment", EXPERIMENT_TAG)
    mlflow.set_tag("mlflow.source.name", "kaggle_notebook_Granite_4_0_Fine_tuning")
    mlflow.set_tag("mlflow.source.type", "KAGGLE_NOTEBOOK")
    mlflow.set_tag("project", "granite-finetuning")

    results = mlflow.evaluate(
        model=anthropic_model,              
        data=eval_df,
        # input_columns="inputs",
        targets="ground_truth",
        model_type="question-answering",
        extra_metrics=[answer_correctness(), latency(),toxicity()],
        evaluator_config={"num_workers": 1}
    )

    print("📊 Aggregated metrics:")
    print(results.metrics)

    print("\n🧾 Evaluation table:")
    display(results.tables["eval_results_table"])

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

  0%|          | 0/150 [00:00<?, ?it/s]

📊 Aggregated metrics:
{'latency/mean': 6.113036284446716, 'latency/variance': 0.5796742057816472, 'latency/p90': 7.013974785804747, 'toxicity/v1/mean': 0.00046503083508772153, 'toxicity/v1/variance': 4.20894633386751e-09, 'toxicity/v1/p90': 0.0006213824381120503, 'toxicity/v1/ratio': 0.0, 'exact_match/v1': 0.0, 'answer_correctness/v1/mean': nan, 'answer_correctness/v1/variance': nan}

🧾 Evaluation table:


,inputs,ground_truth,outputs,latency,token_count,toxicity/v1/score,answer_correctness/v1/score,answer_correctness/v1/justification
0,who was the wicked magician?\n\nprince milan's...,a mighty ruler in the underworld.,"I'd be happy to help you with inputs! However,...",5.857292,161,0.000438,NaN,Failed to score model on payload. Error: OpenA...
1,what did hyacinthia tell the prince to do when...,throw himself at once on the ground and approa...,"I'd be happy to help you with inputs! However,...",5.365036,161,0.000438,NaN,Failed to score model on payload. Error: OpenA...
2,how will the thirtieth duck feel when the prin...,grateful.,"I'd be happy to help you with inputs! However,...",5.718827,161,0.000438,NaN,Failed to score model on payload. Error: OpenA...
3,what happened when hyacinthia stamped on the g...,the earth opened and they both sank down into ...,"I'd be happy to help you with inputs! However,...",5.492132,161,0.000438,NaN,Failed to score model on payload. Error: OpenA...
4,why was the king and queen lamenting before th...,they did not want the prince to leave.,"I'd be happy to help you with inputs! However,...",5.631848,161,0.000438,NaN,Failed to score model on payload. Error: OpenA...
...,...,...,...,...,...,...,...,...
145,what did roc say he would challenge sgiath bre...,run a race with him.,"I'd be happy to help you with inputs! However,...",6.048972,160,0.000621,NaN,Failed to score model on payload. Error: OpenA...
146,what did roc's betrothed do when she head abou...,sneered.,"I'd be happy to help you with inputs! However,...",6.508096,161,0.000438,NaN,Failed to score model on payload. Error: OpenA...
147,what happened after roc was sneered by his bet...,"roc went to his beloved friend, aengus of the ...","I'd be happy to help you with inputs! However,...",6.024156,160,0.000621,NaN,Failed to score model on payload. Error: OpenA...
148,what happened after fionn heard roc's story?\n...,fionn repented himself of the indignity put up...,"I'd be happy to help you with inputs! However,...",5.344171,161,0.000438,NaN,Failed to score model on payload. Error: OpenA...


🏃 View run granite4-anthropic-evaluation-v3 at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/6/runs/6f8ab1b6143d4ebd9fa237fb6b26aec4
🧪 View experiment at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/6


[Trace(trace_id=tr-bdd640fb06671ad11c80317fa3b1799d), Trace(trace_id=tr-1a3d1fa7bc8960a923b8c1e9392456de), Trace(trace_id=tr-972a846916419f828b9d2434e465e150), Trace(trace_id=tr-3b8faa1837f8a88b17fc695a07a0ca6e), Trace(trace_id=tr-b74d0fb132e706298fadc1a606cb0fb3), Trace(trace_id=tr-72ff5d2a386ecbe06b65a6a48b8148f6), Trace(trace_id=tr-c241330b01a9e71fde8a774bcf36d58b), Trace(trace_id=tr-47229389571aa8766c307511b2b9437a), Trace(trace_id=tr-1a2a73ed562b0f79c37459eef50bea63), Trace(trace_id=tr-580d7b71d8f564135be6128e18c26797)]